In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("datasets_hi4/train-data.csv",sep=";")

In [3]:
data

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-12,NaN,may-aug 2021,0,0,0,0
1,469488,224631,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,NaN,jan-apr 2021,0,1,0,0
2,348904,104047,EUROPE,GB,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,Strategic Product Family-7,NaN,jan-apr 2021,0,2,0,0
3,1725822,11823,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-672,Product Line-1,Division-1,Customer Segmentation-11,Strategic Product Family-3,NaN,jan-apr 2023,0,0,0,0
4,404781,159924,EUROPE,ES,NL_DC_HLD,Europe Operations,Iberia,Spain,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,Strategic Product Family-7,NaN,jan-apr 2021,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173341,1285020,60735,NAM,US,US_DC_Mechanicsburg,North America Operations,US,USA,reference-6024,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-12,EOL,may-aug 2022,0,0,0,0
3173342,1527076,57934,NAM,US,US_DC_Athens,North America Operations,US,USA,reference-5672,Product Line-4,Division-3,Customer Segmentation-43,Strategic Product Family-9,NaN,sep-dec 2022,0,0,0,0
3173343,306473,61616,NAM,US,US_DC_Mechanicsburg,North America Operations,US,USA,reference-6241,Product Line-4,Division-3,Customer Segmentation-46,Strategic Product Family-12,NaN,jan-apr 2021,0,0,0,0
3173344,205624,205624,EUROPE,FR,FR_DC_Evreux,France Operations,France,France,reference-11174,Product Line-4,Division-3,Customer Segmentation-1,Strategic Product Family-9,NaN,sep-dec 2020,0,1,0,0


In [4]:
data.columns

Index(['index', 'id_product', 'Region', 'Country', 'Site', 'Operations',
       'Zone', 'Cluster', 'Reference proxy', 'Product  Line proxy',
       'Division proxy', 'Customer Persona proxy',
       'Strategic Product Family proxy', 'Product Life cycel status', 'Date',
       'Month 1', 'Month 2', 'Month 3', 'Month 4'],
      dtype='object')

In [5]:
data.isna().sum()

index                                   0
id_product                              0
Region                                  0
Country                                 0
Site                                    0
Operations                              0
Zone                                    0
Cluster                                 0
Reference proxy                         0
Product  Line proxy                     0
Division proxy                          0
Customer Persona proxy                  0
Strategic Product Family proxy          0
Product Life cycel status         2742544
Date                                    0
Month 1                            352330
Month 2                                 0
Month 3                                 0
Month 4                                 0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173346 entries, 0 to 3173345
Data columns (total 19 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   index                           int64 
 1   id_product                      int64 
 2   Region                          object
 3   Country                         object
 4   Site                            object
 5   Operations                      object
 6   Zone                            object
 7   Cluster                         object
 8   Reference proxy                 object
 9   Product  Line proxy             object
 10  Division proxy                  object
 11  Customer Persona proxy          object
 12  Strategic Product Family proxy  object
 13  Product Life cycel status       object
 14  Date                            object
 15  Month 1                         object
 16  Month 2                         object
 17  Month 3                         object
 18  Mo

In [7]:
duplicate_rows = data[data.duplicated()]
if not duplicate_rows.empty:
    data = data.drop_duplicates(keep='first')

In [8]:
data["Month 1"] = data["Month 1"].apply(lambda x: int(x.replace(" ","")) if isinstance(x, str) else x)

data["Month 2"] = data["Month 2"].apply(lambda x: x.replace(" ","")).astype(int)

data["Month 3"] = data["Month 3"].apply(lambda x: x.replace(" ","")).astype(int)

data["Month 4"] = data["Month 4"].apply(lambda x: x.replace(" ","")).astype(int)

def fill_nan_with_mean(row):
    if pd.isna(row['Month 1']):
        return (row['Month 2'] + row['Month 3']) // 2
    else:
        return row['Month 1']

data['Month 1'] = data.apply(fill_nan_with_mean, axis=1)

data['Month 1'] = data['Month 1'].astype(int)

In [9]:
data['start_date'] = data['Date'].apply(lambda x : x.split("-")[0])
data['end_date'] = data['Date'].apply(lambda x : x.split("-")[1][:3])
data['year'] = data["Date"].apply(lambda x: int(x.split(" ")[1]))

In [10]:
def assign_period(month):
    if month == 'jan':
        return 1
    elif month == 'may':
        return 2
    elif month == 'sep':
        return 3
    else:
        return np.nan

data['Period'] = data['start_date'].apply(assign_period)

In [13]:
data

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4,start_date,end_date,year,Period
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,NaN,may-aug 2021,0,0,0,0,may,aug,2021,2
1,469488,224631,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,...,NaN,jan-apr 2021,0,1,0,0,jan,apr,2021,1
2,348904,104047,EUROPE,GB,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,reference-3513,Product Line-3,...,NaN,jan-apr 2021,0,2,0,0,jan,apr,2021,1
3,1725822,11823,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-672,Product Line-1,...,NaN,jan-apr 2023,0,0,0,0,jan,apr,2023,1
4,404781,159924,EUROPE,ES,NL_DC_HLD,Europe Operations,Iberia,Spain,reference-3496,Product Line-3,...,NaN,jan-apr 2021,0,0,0,0,jan,apr,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173317,1445304,221019,CHINA,CN,CN_DC_SHD,China Operations,China & HK,China,reference-17197,Product Line-2,...,NaN,may-aug 2022,0,0,0,0,may,aug,2022,2
3173331,738575,4004,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-309,Product Line-1,...,EOL,sep-dec 2021,0,1,0,0,sep,dec,2021,3
3173333,68026,68026,NAM,US,US_DC_Mechanicsburg,North America Operations,US,USA,reference-7393,Product Line-4,...,NaN,sep-dec 2020,18,1,0,0,sep,dec,2020,3
3173334,1841243,127244,EUROPE,IT,IT_DC_Venaria,Europe Operations,Italy,Italy,reference-8620,Product Line-4,...,NaN,jan-apr 2023,0,1,0,0,jan,apr,2023,1


In [14]:
data["Product Life cycel status"].fillna("Active",inplace=True)
data['Customer Persona proxy'] = data['Customer Persona proxy'].apply(lambda x: x.replace("Customer Segmentation-", ""))
data['Product  Line proxy'] = data['Product  Line proxy'].apply(lambda x: x.replace("Product Line-", ""))
data['Reference proxy'] = data['Reference proxy'].apply(lambda x: x.replace("reference-", ""))
data['Division proxy'] = data['Division proxy'].apply(lambda x: x.replace("Division-", ""))
data['Strategic Product Family proxy'] = data['Strategic Product Family proxy'].apply(lambda x: x.replace("Strategic Product Family-", ""))

In [15]:
data.drop(['index', 'id_product', 'Country', 'Region', 'Site'], axis =1, inplace =True)

In [16]:
data

,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4,start_date,end_date,year,Period
0,Europe Operations,DACH,Germany,13523,4,3,3,12,Active,may-aug 2021,0,0,0,0,may,aug,2021,2
1,China Operations,China & HK,China,12778,4,3,14,9,Active,jan-apr 2021,0,1,0,0,jan,apr,2021,1
2,Europe Operations,UK and Ireland,United Kingdom,3513,3,2,23,7,Active,jan-apr 2021,0,2,0,0,jan,apr,2021,1
3,International Operations,Pacific,Australia,672,1,1,11,3,Active,jan-apr 2023,0,0,0,0,jan,apr,2023,1
4,Europe Operations,Iberia,Spain,3496,3,2,14,7,Active,jan-apr 2021,0,0,0,0,jan,apr,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173317,China Operations,China & HK,China,17197,2,2,28,5,Active,may-aug 2022,0,0,0,0,may,aug,2022,2
3173331,International Operations,Pacific,Australia,309,1,1,8,3,EOL,sep-dec 2021,0,1,0,0,sep,dec,2021,3
3173333,North America Operations,US,USA,7393,4,3,4,9,Active,sep-dec 2020,18,1,0,0,sep,dec,2020,3
3173334,Europe Operations,Italy,Italy,8620,4,3,4,12,Active,jan-apr 2023,0,1,0,0,jan,apr,2023,1


In [17]:
data.isna().sum()

Operations                        0
Zone                              0
Cluster                           0
Reference proxy                   0
Product  Line proxy               0
Division proxy                    0
Customer Persona proxy            0
Strategic Product Family proxy    0
Product Life cycel status         0
Date                              0
Month 1                           0
Month 2                           0
Month 3                           0
Month 4                           0
start_date                        0
end_date                          0
year                              0
Period                            0
dtype: int64